[![AnalyticsDojo](https://github.com/rpi-techfundamentals/fall2018-materials/blob/master/fig/final-logo.png?raw=1)](http://rpi.analyticsdojo.com)
<center><h1>Pytorch - Revisiting Titanic</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>


In [25]:
!pip install torch torchvision

In [26]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn import preprocessing

train= pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')



In [27]:
#Print to standard output, and see the results in the "log" section below after running your script
train.head()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan


In [28]:
#This replaces the Null Names with the variable "Nameless"
train.loc[train.Name.isnull(), "Name"]='Unnamed'
test.loc[test.Name.isnull(), "Name"]='Unnamed'

#This looks through Name and creates a new column called NameCategory and encodes whether the animal is "Named" or "Unnamed"
# train['NameCategory'] = train.Name.apply(lambda x: 0 if x == 'Unnamed' else 1)
# test['NameCategory'] = test.Name.apply(lambda x: 0 if x == 'Unnamed' else 1)
train['NameCategory'] = train.Name.apply(lambda x: 0 if x == 'Unnamed' else 1)
test['NameCategory'] = test.Name.apply(lambda x: 0 if x == 'Unnamed' else 1)

In [29]:
#Evaluate SexuponOutcome
train.loc[train.SexuponOutcome.isnull(), "SexuponOutcome"]='Unknown'
test.loc[test.SexuponOutcome.isnull(), "SexuponOutcome"]='Unknown'

#Extracts whether animal is Female or Male
def clean_sex(x):
    if 'Female' in x:
        return 0
    elif 'Male' in x:
        return 1 
    else:
        return 1
train['Sex'] = None #creates "Sex" column
train['Sex'] = train['SexuponOutcome'].apply(clean_sex)

test['Sex'] = None #creates "Sex" column
test['Sex'] = test['SexuponOutcome'].apply(clean_sex)

#Extracts if anmial is fixed or not
def clean_fixed(x):
    if 'Spay' in x:
        return 'Fixed'
    elif 'Neuter' in x:
        return 'Fixed'
    elif 'Intact' in x:
        return 'Intact'
    else:
        return 'Fixed'
train['FixedIntact'] = None #creates "IsFixed" column
train['FixedIntact'] = train['SexuponOutcome'].apply(clean_fixed)

test['FixedIntact'] = None #creates "IsFixed" column
test['FixedIntact'] = test['SexuponOutcome'].apply(clean_fixed)

In [30]:
#Function that converts all AgeuponOutcome to a consistent unit (ie. years) and removes entires with null Ages
train.loc[train.AgeuponOutcome.isnull(), 'AgeuponOutcome'] = '0 days'
test.loc[test.AgeuponOutcome.isnull(), 'AgeuponOutcome'] = '0 days'

#Scales AgeuponOutcome into standard unit of days
def scale_dates(x):
    if 'year' in x:
        return int(x.split(' ')[0])*365
    elif 'month' in x:
        return int(x.split(' ')[0])*12
    elif 'week' in x:
        return int(x.split(' ')[0])*7
    elif 'day' in x:
        return int(x.split(' ')[0])
    else:
        0
        
train['AgeinDays'] = 0 #Creates "AgeinDays" column
train['AgeinDays']=train['AgeuponOutcome'].apply(scale_dates)

test['AgeinDays'] = 0 #Creates "AgeinDays" column
test['AgeinDays']=test['AgeuponOutcome'].apply(scale_dates)

#Groups ages into 4 categories - Baby, Young adult, Adult & Senior
def category_age(x):
    if x <= 365: #less than or equal to 1 year
        return 'Baby'
    elif 365 < x <= 1095: #greater than 1 year or equal to 3 years
        return 'Young adult' 
    elif 1095 < x <= 1825: #greater than 3 years or equal to 5 years
        return 'Adult'
    else: # greater than 5 years
        return 'Senior'
    
train['AgeCategory'] = 0 #creates "AgeCategory" column
train['AgeCategory'] = train['AgeinDays'].apply(category_age)

test['AgeCategory'] = 0 #creates "AgeCategory" column
test['AgeCategory'] = test['AgeinDays'].apply(category_age)

In [31]:
import datetime
def transform_date(df):
    date = pd.to_datetime(df.DateTime)
    df['Hour'] = date.dt.hour
    df['WeekDay'] = date.dt.weekday
    df['Month'] = date.dt.month
    df['Year'] = date.dt.year

transform_date(train)
transform_date(test)

#Categorizes months by seasons
def category_season(x):
    if x == 12: #December
        return 'Winter'
    elif 1<= x <= 2: #January, Feburary
        return 'Winter'
    elif 3 <= x <= 5: #March, April, May
        return 'Spring'
    elif 6 <= x <= 8: #June, July, August
        return 'Summer'
    else: #September, October, November
        return 'Fall'
    
train['Season'] = None #Creates Season column
train['Season'] = train['Month'].apply(category_season)

test['Season'] = None #Creates Season column
test['Season'] = test['Month'].apply(category_season)

In [32]:
#Luckly there are no nulls in Breed, but I included these lines in the off chance there is a null
# train.loc[train.Breed.isnull(), 'Breed'] = 'Unknown'
# test.loc[test.Breed.isnull(), 'Breed'] = 'Unknown'

#Column indicators
train['IsMix'] = train['Breed'].apply(lambda x: 1 if 'Mix' in x else 0) #Creates column indicator if animal is defined as "Mix"
train['IsMultipleBreeds'] = train['Breed'].apply(lambda x: 1 if '/' in x else 0) #Creates column indicator if animal has two breeds listed

test['IsMix'] = test['Breed'].apply(lambda x: 1 if 'Mix' in x else 0) #Creates column indicator if animal is defined as "Mix"
test['IsMultipleBreeds'] = test['Breed'].apply(lambda x: 1 if '/' in x else 0) #Creates column indicator if animal has two breeds listed

In [33]:
#Luckly there are no nulls in Color, but I included these lines in the off chance there is a null
# train.loc[train.Color.isnull(), 'Color'] = 'Unknown'
# test.loc[test.Color.isnull(), 'Color'] = 'Unknown'

#Column indicators
train['IsMultipleColors'] = train['Color'].apply(lambda x: 1 if '/' in x else 0) #Creates column indicator if animal has more than one color listed
test['IsMultipleColors'] = test['Color'].apply(lambda x: 1 if '/' in x else 0) #Creates column indicator if animal has more than one color listed

In [34]:
# def category_breed(df):
#     df['IsMix'] = df['Breed'].apply(lambda x: 1 if 'Mix' in x else 0)
#     breeds = train['Breed'].apply(lambda x: x.split('/'))
#     df['Breed1'] = breeds.apply(lambda x: x[0].replace(' Mix', ''))
#     df['Breed2'] = breeds.apply(lambda x: x[1] if len(x)>1 else x[0].replace(' Mix', ''))
#     df['MultipleBreeds'] = df['Breed'].apply(lambda x: 1 if '/' in x else 0)
# category_breed(train)
# category_breed(test)

In [35]:
# #Used to split 
# def category_color(df):
#     color1 = ""
#     color2 = ""
#     colors = df['Color'].apply(lambda x: x.split('/'))
#     for i in range(len(colors)):
#         if len(colors[i]) == 1:
#             hit = False
#             for j in range(len(colors[i])):
#                 for k in range(len(colors[i][j])):
#                     if colors[i][j][k] == " ":
#                         hit = True
#                         color_split_set = colors[i][j].split()
#                         df.loc[i,'Color1'] = color_split_set[0]
#                         df.loc[i,'Color2'] = color_split_set[0]
#                 if hit == False:
#                     df.loc[i,'Color1'] = colors[i][j]
#                     df.loc[i,'Color2'] = colors[i][j]
#         else:
#             hit1 = False
#             hit2 = False

#             for j in range(len(colors[i])):
#                 if j == 0:
#                     for k in range(len(colors[i][j])):
#                         if colors[i][j][k] == " ":
#                             hit1 = True
#                             color_split_set = colors[i][j].split()
#                             df.loc[i,'Color1'] = color_split_set[0]
                        
#                     if hit1 == False:
#                         df.loc[i,'Color1'] = colors[i][j]
                        
#                 else:
#                     for k in range(len(colors[i][j])):
#                         if colors[i][j][k] == " ":
#                             hit2 = True
#                             color_split_set = colors[i][j].split()
#                             df.loc[i,'Color2'] = color_split_set[0]
#                     if hit2 == False:
#                         df.loc[i,'Color2'] = colors[i][j]               
# category_color(train)
# category_color(test)

In [53]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encode = LabelEncoder()
train.loc[:,'OutcomeType'] = encode.fit_transform(train['OutcomeType'])

In [54]:
#Drop some colums that won't be modeled. 
train_min = train.drop(['AnimalID', 'Name', 'SexuponOutcome', 'DateTime', 'OutcomeSubtype', 'AgeuponOutcome', 'Breed', 'Color'], axis = 1)
test_min= test.drop(['ID', 'Name', 'DateTime', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'], axis = 1)
train_min.head()

,OutcomeType,AnimalType,NameCategory,Sex,FixedIntact,AgeinDays,AgeCategory,Hour,WeekDay,Month,Year,Season,IsMix,IsMultipleBreeds,IsMultipleColors
0,3,Dog,1,1,Fixed,365,Baby,18,2,2,2014,Winter,1,0,1
1,2,Cat,1,0,Fixed,365,Baby,12,6,10,2013,Fall,1,0,0
2,0,Dog,1,1,Fixed,730,Young adult,12,5,1,2015,Winter,1,0,1
3,4,Cat,0,1,Intact,21,Baby,19,4,7,2014,Summer,1,0,0
4,4,Dog,0,1,Fixed,730,Young adult,12,4,11,2013,Fall,0,1,0


In [55]:
#Adding .values will make whatever comes out into a numpy array. 
X_train = train_min.iloc[:, 1:15].values
Y_train = train_min.iloc[:, 0].values
X_test = test_min.iloc[:, 0:14].values

print(X_test[1], X_train[1])
print(len(X_test[1]), len(X_train[1]))

['Dog' 1 0 'Fixed' 730 'Young adult' 17 5 7 2014 'Summer' 0 1 1] ['Cat' 1 0 'Fixed' 365 'Baby' 12 6 10 2013 'Fall' 1 0 0]
14 14


In [56]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# Previously we used get dummies (part of pandas)
#Here we will first transform string labels to numeric categories. 
labelencoder_X_1 = LabelEncoder()
X_train[:, 0] = labelencoder_X_1.fit_transform(X_train[:, 0]) #AnimalType
X_train[:, 3] = labelencoder_X_1.fit_transform(X_train[:, 3]) #Fixed/Intact
X_train[:, 5] = labelencoder_X_1.fit_transform(X_train[:, 5]) #AgeCategory
X_train[:, 10] = labelencoder_X_1.fit_transform(X_train[:, 10]) #Breed1
# X_train[:, 11] = labelencoder_X_1.fit_transform(X_train[:, 11]) #Breed2
# X_train[:, 13] = labelencoder_X_1.fit_transform(X_train[:, 13]) #Color1
# X_train[:, 14] = labelencoder_X_1.fit_transform(X_train[:, 14]) #Color2

labelencoder_X_2 = LabelEncoder()
X_test[:, 0] = labelencoder_X_2.fit_transform(X_test[:, 0])
X_test[:, 3] = labelencoder_X_2.fit_transform(X_test[:, 3])
X_test[:, 5] = labelencoder_X_1.fit_transform(X_test[:, 5]) 
X_test[:, 10] = labelencoder_X_2.fit_transform(X_test[:, 10])
# X_test[:, 11] = labelencoder_X_2.fit_transform(X_test[:, 11])
# X_test[:, 13] = labelencoder_X_2.fit_transform(X_test[:, 13])
# X_test[:, 14] = labelencoder_X_1.fit_transform(X_test[:, 14])
X_test.shape

(11456, 14)

In [57]:
print(pd.DataFrame(X_test).isna().sum())
print(pd.DataFrame(X_test).isna().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [58]:
# Converting numeric categories values to one-hot representation
one_hot_encoder= OneHotEncoder(categorical_features = [0, 3, 5, 10])
X_train = one_hot_encoder.fit_transform(X_train).toarray()
X_test = one_hot_encoder.fit_transform(X_test).toarray()


In [59]:
#Double check no missing values or values that aren't numeric.
print(np.isnan(X_train).sum(),np.isnan(X_test).sum())
print(X_train[1])

0 0
[1.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 3.650e+02 1.200e+01 6.000e+00 1.000e+01 2.013e+03 1.000e+00 0.000e+00
 0.000e+00]


In [60]:
#Split the data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.2)

In [61]:
y_val.shape

(5346,)

In [66]:
#Define the model 
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #Note that 17 is the number of columns in the input matrix. 
        self.fc1 = nn.Linear(22, 50)
        #270 is arbitrary, but needs to be consistent.  2 is the number of classes in the output (died/survived)
        self.fc2 = nn.Linear(50, 5)
        
    def forward(self, x):
       # x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)  #signmoid flattens to probability 0-1. 
        return x
    
net = Net()

In [67]:
#Define training
batch_size = 50
num_epochs = 100
learning_rate = 0.01
batch_no = len(x_train) // batch_size

In [68]:
#define loss function and optimizer 
#Adam is a specific flavor of gradient decent which is typically better
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [69]:
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0
for epoch in range(num_epochs):
    x_train, y_train = shuffle(x_train, y_train)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(x_train[start:end]))
        labels = Variable(torch.LongTensor(y_train[start:end]))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0

Epoch 1 loss:  590.8637751340866
Epoch 2 loss:  590.6007400751114
Epoch 3 loss:  590.5850187540054
Epoch 4 loss:  590.605019569397
Epoch 5 loss:  590.5850186347961
Epoch 6 loss:  590.6250176429749
Epoch 7 loss:  590.6050173044205
Epoch 8 loss:  590.6250177621841
Epoch 9 loss:  590.5050183534622
Epoch 10 loss:  590.625018954277
Epoch 11 loss:  590.6250176429749
Epoch 12 loss:  590.5450193881989
Epoch 13 loss:  590.5650185346603
Epoch 14 loss:  590.5650191307068
Epoch 15 loss:  590.6050170660019
Epoch 16 loss:  590.5250185728073
Epoch 17 loss:  590.6050189733505
Epoch 18 loss:  590.5650174617767
Epoch 19 loss:  590.5650187730789
Epoch 20 loss:  590.6050190925598
Epoch 21 loss:  590.5650182962418
Epoch 22 loss:  590.5850192308426
Epoch 23 loss:  590.5650197267532
Epoch 24 loss:  590.5850172042847
Epoch 25 loss:  590.6050182580948
Epoch 26 loss:  590.5650171041489
Epoch 27 loss:  590.6250168085098
Epoch 28 loss:  590.5813108682632
Epoch 29 loss:  590.5650181770325
Epoch 30 loss:  590.58501

In [70]:
#This is a little bit tricky to get the resulting prediction.  
def calculate_accuracy(x,y=[]):
  # Evaluate the model with the test set. 
  test_var = Variable(torch.FloatTensor(x), requires_grad=True)
  with torch.no_grad():   
      result = net(test_var) #This outputs the probability for each class.
  values, labels = torch.max(result, 1)
  if len(y) != 0:
      num_right = np.sum(labels.data.numpy() == y)
      print('Accuracy {:.2f}'.format(num_right / len(y)), "for a total of ", len(y), "records")
  else:
      print("returning predictions")
      return labels.data.numpy()
 
  


In [71]:
# Check out the accuracy. 
calculate_accuracy(x_train, y_train)
calculate_accuracy(x_val, y_val)
predictions=calculate_accuracy(X_test)
len(predictions)
predictions

Accuracy 0.35 for a total of  21383 records
Accuracy 0.35 for a total of  5346 records
returning predictions


array([4, 4, 4, ..., 4, 4, 4])

In [72]:
#Writing to File
submission=pd.DataFrame(test.loc[:,['ID']])
submission['OutcomeType']=predictions
#Any files you save will be available in the output tab below

submission.to_csv('Pytorchsubmission.csv', index=False)

 

In [ ]:
#from google.colab import files
#files.download('submission.csv')

In [73]:
# Evaluate the model
test_var = Variable(torch.FloatTensor(x_val), requires_grad=True)
with torch.no_grad():
    result = net(test_var)
values, labels = torch.max(result, 1)
num_right = np.sum(labels.data.numpy() == y_val)
print('Accuracy {:.2f}'.format(num_right / len(y_val)))

Accuracy 0.35
